# Rolling Window Features

Rolling statistics smooth noise and summarize local behavior. They are used for trend extraction,
volatility tracking, and building robust ML features.


In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

np.random.seed(11)

n = 160
t = np.arange(n)
y = 4 + 0.04 * t + 1.5 * np.sin(2 * np.pi * t / 30) + np.random.normal(0, 0.7, n)
series = pd.Series(y, index=pd.RangeIndex(n), name="y")

series.head()


## Definitions

Rolling mean with window $w$:

$$m_t = 
\frac{1}{w} \sum_{i=0}^{w-1} y_{t-i}$$

Rolling standard deviation:

$$s_t = 
\sqrt{\frac{1}{w-1} \sum_{i=0}^{w-1} (y_{t-i} - m_t)^2}$$


In [ ]:
w_short = 7
w_long = 30

df_roll = pd.DataFrame({"y": series})
df_roll[f"roll_mean_{w_short}"] = series.rolling(w_short).mean()
df_roll[f"roll_mean_{w_long}"] = series.rolling(w_long).mean()
df_roll[f"roll_std_{w_short}"] = series.rolling(w_short).std()
df_roll.head(12)


## Trend smoothing (short vs long windows)

Short windows respond quickly but are noisy; long windows are smoother but lag more.


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=series.index, y=series, mode="lines", name="y_t", line=dict(color="#1f77b4")))
fig.add_trace(go.Scatter(x=series.index, y=df_roll[f"roll_mean_{w_short}"], mode="lines", name=f"Rolling mean ({w_short})", line=dict(color="#ff7f0e")))
fig.add_trace(go.Scatter(x=series.index, y=df_roll[f"roll_mean_{w_long}"], mode="lines", name=f"Rolling mean ({w_long})", line=dict(color="#2ca02c")))
fig.update_layout(title="Rolling mean smoothing", xaxis_title="t", yaxis_title="value", height=420)
fig.show()


## Rolling volatility

The rolling standard deviation tracks local variability and is a common volatility feature.


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=series.index, y=df_roll[f"roll_std_{w_short}"], mode="lines", name=f"Rolling std ({w_short})", line=dict(color="#d62728")))
fig.update_layout(title="Rolling standard deviation", xaxis_title="t", yaxis_title="std", height=360)
fig.show()


## sktime mapping (where this lives)

For feature engineering in sktime, `WindowSummarizer` creates rolling statistics over defined
windows, and it can be combined with other transformers in a pipeline.
